# Manage Conversation History

Long conversations can exceed the LLM's context window. Common solutions are:

* [Trimming](#trim-messages) — Remove first or last N messages (before calling LLM)
* [Summarization](#summarize-messages) — Summarize earlier messages in the history and replace them with a summary
* [Deleting messages](#delete-messages) - Delete messages from LangGraph state permanently
* custom strategies (e.g., message filtering, etc.)

This allows the agent to keep track of the conversation without exceeding the LLM's context window.

In [ ]:
# hide-cell
%%capture --no-stderr
%pip install --quiet -U langgraph "langchain[anthropic]"

In [36]:
# hide-cell
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("ANTHROPIC_API_KEY")

ANTHROPIC_API_KEY:  ········


## Process Model Input

### In a model-calling node

Use this if you need to process input to the LLM. In most cases, this will be sufficient.

```python
def process_messages(messages: list[AnyMessage]):
    # keep only the last message
    return messages[-1:]

def call_model(state: MessagesState):
    # highlight-next-line
    messages = process_messages(state["messages"])
    response = model.invoke(messages)
    return {"messages": [response]}


builder = StateGraph(MessagesState)
builder.add_node(call_model)
...
```

??? example "Full example: process messages in a model-calling node"

    ```python
    from langchain.chat_models import init_chat_model
    from langchain_core.messages import AnyMessage
    
    from langgraph.graph import MessagesState, StateGraph, START
    from langgraph.checkpoint.memory import InMemorySaver
    
    def process_messages(messages: list[AnyMessage]):
        # keep only the last message
        return messages[-1:]
    
    def call_model(state: MessagesState):
        # highlight-next-line
        messages = process_messages(state["messages"])
        response = model.invoke(messages)
        return {"messages": [response]}
    
    
    # highight-next-line
    checkpointer = InMemorySaver()
    model = init_chat_model("anthropic:claude-3-5-haiku-latest")
    
    builder = StateGraph(MessagesState)
    builder.add_node(call_model)
    builder.add_edge(START, "call_model")
    # highight-next-line
    app = builder.compile(checkpointer=checkpointer)
    
    config = {"configurable": {"thread_id": "1"}}
    for event in app.stream(
        {"messages": [{"role": "user", "content": "hi! I'm bob"}]},
        config,
        stream_mode="values"
    ):
        event["messages"][-1].pretty_print()
    
    # This will now not remember the previous messages
    # because we set `messages[-1:]` in the process_messages function
    for event in app.stream(
        {"messages": [{"role": "user", "content": "what's my name?"}]},
        config,
        stream_mode="values"
    ):
        event["messages"][-1].pretty_print()
    ```

    ```
    ================================ Human Message =================================
    
    hi! I'm bob
    ================================== Ai Message ==================================
    
    Hi Bob! How are you doing today? Is there anything I can help you with?
    ================================ Human Message =================================
    
    what's my name?
    ================================== Ai Message ==================================
    
    I apologize, but you haven't told me your name yet. Could you please introduce yourself?
    ```

In [96]:
# hide-cell
from langchain.chat_models import init_chat_model
from langchain_core.messages import AnyMessage

from langgraph.graph import MessagesState, StateGraph, START
from langgraph.checkpoint.memory import InMemorySaver

def process_messages(messages: list[AnyMessage]):
    # keep only the last message
    return messages[-1:]

def call_model(state: MessagesState):
    # highlight-next-line
    messages = process_messages(state["messages"])
    response = model.invoke(messages)
    return {"messages": [response]}


# highight-next-line
checkpointer = InMemorySaver()
model = init_chat_model("anthropic:claude-3-5-haiku-latest")

builder = StateGraph(MessagesState)
builder.add_node(call_model)
builder.add_edge(START, "call_model")
# highight-next-line
app = builder.compile(checkpointer=checkpointer)

config = {"configurable": {"thread_id": "1"}}
for event in app.stream(
    {"messages": [{"role": "user", "content": "hi! I'm bob"}]},
    config,
    stream_mode="values"
):
    event["messages"][-1].pretty_print()

# This will now not remember the previous messages
# because we set `messages[-1:]` in the process_messages function
for event in app.stream(
    {"messages": [{"role": "user", "content": "what's my name?"}]},
    config,
    stream_mode="values"
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

hi! I'm bob
================================== Ai Message ==================================

Hi Bob! How are you doing today? Is there anything I can help you with?
================================ Human Message =================================

what's my name?
================================== Ai Message ==================================

I apologize, but you haven't told me your name yet. Could you please introduce yourself?


### In a separate node

Use this if you need to:

* stream and checkpoint message processing as a separate graph step
* **both** preprocess inputs to the LLM **and** update the graph state. This is useful when summarizing message history, where `process_messages` step needs to return both the LLM input messages and a running summary information.

```python
from typing_extensions import TypedDict

class OverallState(MessagesState):
    some_key: str

class LLMInputState(TypedDict):
    # highlight-next-line
    llm_input_messages: list[AnyMessage]

def process_messages(state: MessagesState):
    return {
        # keep only the last message
        "llm_input_messages": state["messages"][-1:],  # (1)!
        "some_key": "some_value"  # (2)!
    }

# highlight-next-line
def call_model(state: LLMInputState):  # (3)!
    response = model.invoke(state["llm_input_messages"])
    return {"messages": response}

builder = StateGraph(OverallState)
# highlight-next-line
builder.add_node(process_messages)
builder.add_node(call_model)
...
```

1. `llm_input_messages` will be visible only to the `call_model` node and won't be added to overall graph state.
2. `some_key` will be added to the **overall** graph state.
3. `call_model` uses a private input schema, different from the `OverallState`. This allows us to change the input to `call_model` node without affecting the **overall** graph state.

??? example "Full example: process messages in a separate node"

    ```python
    from typing_extensions import TypedDict
    
    class OverallState(MessagesState):
        some_key: str
    
    
    class LLMInputState(TypedDict):
        # highlight-next-line
        llm_input_messages: list[AnyMessage]
    
    
    def process_messages(state: MessagesState):
        return {
            # keep only the last message
            "llm_input_messages": state["messages"][-1:],  # (1)!
            "some_key": "some_value"  # (2)!
        }
    
    
    # highlight-next-line
    def call_model(state: LLMInputState):  # (3)!
        response = model.invoke(state["llm_input_messages"])
        return {"messages": response}
    
    
    checkpointer = InMemorySaver()
    model = init_chat_model("anthropic:claude-3-5-haiku-latest")
    
    builder = StateGraph(OverallState)
    # highlight-next-line
    builder.add_node(process_messages)
    builder.add_node(call_model)
    builder.add_edge(START, "process_messages")
    builder.add_edge("process_messages", "call_model")
    
    app = builder.compile(checkpointer=checkpointer)
    
    config = {"configurable": {"thread_id": "1"}}
    for event in app.stream(
        {"messages": [{"role": "user", "content": "hi! I'm bob"}]},
        config,
        stream_mode="updates"
    ):
        print(event)
        print("\n")
    
    # This will now not remember the previous messages
    # because we set `messages[-1:]` in the filter_messages function
    for event in app.stream(
        {"messages": [{"role": "user", "content": "what's my name?"}]},
        config,
        stream_mode="updates"
    ):
        print(event)
        print("\n")
    ```
    
    1. `llm_input_messages` will be visible only to the `call_model` node and won't be added to overall graph state.
    2. `some_key` will be added to the **overall** graph state.
    3. `call_model` uses a private input schema, different from the `OverallState`. This allows us to change the input to `call_model` node without affecting the **overall** graph state.

    ```
    {'process_messages': {'llm_input_messages': [HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}, id='6eb8c38a-7f7c-4c7e-98ab-2e01e3e860a9')], 'some_key': 'some_value'}}


    {'call_model': {'messages': AIMessage(content='Hi Bob! How are you doing today? Is there anything I can help you with?', additional_kwargs={}, response_metadata={'id': 'msg_01SoZtnvMLsSJCQdL6ajXFK8', 'model': 'claude-3-5-haiku-20241022', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 12, 'output_tokens': 21}, 'model_name': 'claude-3-5-haiku-20241022'}, id='run-d8998b94-4f49-494b-9b91-85f676cba22a-0', usage_metadata={'input_tokens': 12, 'output_tokens': 21, 'total_tokens': 33, 'input_token_details': {'cache_read': 0, 'cache_creation': 0}})}}
    
    
    {'process_messages': {'llm_input_messages': [HumanMessage(content="what's my name?", additional_kwargs={}, response_metadata={}, id='9a38c63e-f23d-4c38-a06d-7d7c74ade462')], 'some_key': 'some_value'}}
    
    
    {'call_model': {'messages': AIMessage(content="I apologize, but I don't know your name. I can't automatically know personal details about you unless you've previously told me. Could you introduce yourself?", additional_kwargs={}, response_metadata={'id': 'msg_01EZbuZrc65PKy8gGF4TcB3i', 'model': 'claude-3-5-haiku-20241022', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 12, 'output_tokens': 36}, 'model_name': 'claude-3-5-haiku-20241022'}, id='run-b375691b-078a-4a87-a642-8bf72a96c198-0', usage_metadata={'input_tokens': 12, 'output_tokens': 36, 'total_tokens': 48, 'input_token_details': {'cache_read': 0, 'cache_creation': 0}})}}
    ```

In [95]:
# hide-cell
from typing_extensions import TypedDict

class OverallState(MessagesState):
    some_key: str


class LLMInputState(TypedDict):
    # highlight-next-line
    llm_input_messages: list[AnyMessage]


def process_messages(state: MessagesState):
    return {
        # keep only the last message
        "llm_input_messages": state["messages"][-1:],
        "some_key": "some_value"
    }


# highlight-next-line
def call_model(state: LLMInputState):
    response = model.invoke(state["llm_input_messages"])
    return {"messages": response}


checkpointer = InMemorySaver()
model = init_chat_model("anthropic:claude-3-5-haiku-latest")

builder = StateGraph(OverallState)
# highlight-next-line
builder.add_node(process_messages)
builder.add_node(call_model)
builder.add_edge(START, "process_messages")
builder.add_edge("process_messages", "call_model")

app = builder.compile(checkpointer=checkpointer)

config = {"configurable": {"thread_id": "1"}}
for event in app.stream(
    {"messages": [{"role": "user", "content": "hi! I'm bob"}]},
    config,
    stream_mode="updates"
):
    print(event)
    print("\n")

# This will now not remember the previous messages
# because we set `messages[-1:]` in the filter_messages function
for event in app.stream(
    {"messages": [{"role": "user", "content": "what's my name?"}]},
    config,
    stream_mode="updates"
):
    print(event)
    print("\n")

{'process_messages': {'llm_input_messages': [HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}, id='6eb8c38a-7f7c-4c7e-98ab-2e01e3e860a9')], 'some_key': 'some_value'}}


{'call_model': {'messages': AIMessage(content='Hi Bob! How are you doing today? Is there anything I can help you with?', additional_kwargs={}, response_metadata={'id': 'msg_01SoZtnvMLsSJCQdL6ajXFK8', 'model': 'claude-3-5-haiku-20241022', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 12, 'output_tokens': 21}, 'model_name': 'claude-3-5-haiku-20241022'}, id='run-d8998b94-4f49-494b-9b91-85f676cba22a-0', usage_metadata={'input_tokens': 12, 'output_tokens': 21, 'total_tokens': 33, 'input_token_details': {'cache_read': 0, 'cache_creation': 0}})}}


{'process_messages': {'llm_input_messages': [HumanMessage(content="what's my name?", additional_kwargs={}, response_metadata={}, id='9a38c63e-f23d-4c38-a06d-7d

## Strategies

### Trim messages

TODO: populate this

---

### Delete messages

To delete messages from the graph state, you can use the `RemoveMessage`.

* Remove specific messages:

    ```python
    # highlight-next-line
    from langchain_core.messages import RemoveMessage
    
    def delete_messages(state):
        messages = state["messages"]
        if len(messages) > 2:
            # remove the earliest two messages
            # highlight-next-line
            return {"messages": [RemoveMessage(id=m.id) for m in messages[:2]]}
    ```

* Remove **all** messages:
    
    ```python
    # highlight-next-line
    from langgraph.graph.message import REMOVE_ALL_MESSAGES
    
    def delete_messages(state):
        # highlight-next-line
        return {"messages": [RemoveMessage(id=REMOVE_ALL_MESSAGES)]}
    ```

!!! important "`add_messages` reducer"

    For `RemoveMessage` to work, you need to use a state key with [`add_messages`][langgraph.graph.message.add_messages] [reducer](../../../concepts/low_level#messagesstate), like [`MessagesState`](../../../concepts/low_level#messagesstate)

!!! warning "Valid message history"

    When deleting messages, **make sure** that the resulting message history is valid. Check the limitations of the LLM provider you're using. For example:
    
    * some providers expect message history to start with a `user` message
    * most providers require `assistant` messages with tool calls to be followed by corresponding `tool` result messages.

??? example "Full example: delete messages"

    ```python
    # highlight-next-line
    from langchain_core.messages import RemoveMessage
    
    def delete_messages(state):
        messages = state["messages"]
        if len(messages) > 2:
            # remove the earliest two messages
            # highlight-next-line
            return {"messages": [RemoveMessage(id=m.id) for m in messages[:2]]}
    
    def call_model(state: MessagesState):
        response = model.invoke(state["messages"])
        return {"messages": response}
    
    builder = StateGraph(MessagesState)
    builder.add_sequence([call_model, delete_messages])
    builder.add_edge(START, "call_model")
    
    checkpointer = InMemorySaver()
    app = builder.compile(checkpointer=checkpointer)
    
    for event in app.stream(
        {"messages": [{"role": "user", "content": "hi! I'm bob"}]},
        config,
        stream_mode="values"
    ):
        print([(message.type, message.content) for message in event["messages"]])
    
    for event in app.stream(
        {"messages": [{"role": "user", "content": "what's my name?"}]},
        config,
        stream_mode="values"
    ):
        print([(message.type, message.content) for message in event["messages"]])
    ```

    ```
    [('human', "hi! I'm bob")]
    [('human', "hi! I'm bob"), ('ai', 'Hi Bob! How are you doing today? Is there anything I can help you with?')]
    [('human', "hi! I'm bob"), ('ai', 'Hi Bob! How are you doing today? Is there anything I can help you with?'), ('human', "what's my name?")]
    [('human', "hi! I'm bob"), ('ai', 'Hi Bob! How are you doing today? Is there anything I can help you with?'), ('human', "what's my name?"), ('ai', 'Your name is Bob.')]
    [('human', "what's my name?"), ('ai', 'Your name is Bob.')]
    ```

In [114]:
# hide-cell
from langchain_core.messages import RemoveMessage

def delete_messages(state):
    messages = state["messages"]
    if len(messages) > 2:
        # remove the earliest two messages
        return {"messages": [RemoveMessage(id=m.id) for m in messages[:2]]}

def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}

builder = StateGraph(MessagesState)
builder.add_sequence([call_model, delete_messages])
builder.add_edge(START, "call_model")

checkpointer = InMemorySaver()
app = builder.compile(checkpointer=checkpointer)

for event in app.stream(
    {"messages": [{"role": "user", "content": "hi! I'm bob"}]},
    config,
    stream_mode="values"
):
    print([(message.type, message.content) for message in event["messages"]])

for event in app.stream(
    {"messages": [{"role": "user", "content": "what's my name?"}]},
    config,
    stream_mode="values"
):
    print([(message.type, message.content) for message in event["messages"]])

[('human', "hi! I'm bob")]
[('human', "hi! I'm bob"), ('ai', 'Hi Bob! How are you doing today? Is there anything I can help you with?')]
[('human', "hi! I'm bob"), ('ai', 'Hi Bob! How are you doing today? Is there anything I can help you with?'), ('human', "what's my name?")]
[('human', "hi! I'm bob"), ('ai', 'Hi Bob! How are you doing today? Is there anything I can help you with?'), ('human', "what's my name?"), ('ai', 'Your name is Bob.')]
[('human', "what's my name?"), ('ai', 'Your name is Bob.')]


### Summarize messages

An effective strategy for handling long conversation history is to summarize earlier messages once they reach a certain threshold.

#### TODO: rewrite / simplify this

In [25]:
from typing import Literal

from langchain_anthropic import ChatAnthropic
from langchain_core.messages import SystemMessage, RemoveMessage, HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState, StateGraph, START, END

memory = MemorySaver()


# We will add a `summary` attribute (in addition to `messages` key,
# which MessagesState already has)
class State(MessagesState):
    summary: str


# We will use this model for both the conversation and the summarization
model = ChatAnthropic(model_name="claude-3-haiku-20240307")


# Define the logic to call the model
def call_model(state: State):
    # If a summary exists, we add this in as a system message
    summary = state.get("summary", "")
    if summary:
        system_message = f"Summary of conversation earlier: {summary}"
        messages = [SystemMessage(content=system_message)] + state["messages"]
    else:
        messages = state["messages"]
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}


# We now define the logic for determining whether to end or summarize the conversation
def should_continue(state: State) -> Literal["summarize_conversation", END]:
    """Return the next node to execute."""
    messages = state["messages"]
    # If there are more than six messages, then we summarize the conversation
    if len(messages) > 6:
        return "summarize_conversation"
    # Otherwise we can just end
    return END


def summarize_conversation(state: State):
    # First, we summarize the conversation
    summary = state.get("summary", "")
    if summary:
        # If a summary already exists, we use a different system prompt
        # to summarize it than if one didn't
        summary_message = (
            f"This is summary of the conversation to date: {summary}\n\n"
            "Extend the summary by taking into account the new messages above:"
        )
    else:
        summary_message = "Create a summary of the conversation above:"

    messages = state["messages"] + [HumanMessage(content=summary_message)]
    response = model.invoke(messages)
    # We now need to delete messages that we no longer want to show up
    # I will delete all but the last two messages, but you can change this
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]
    return {"summary": response.content, "messages": delete_messages}


# Define a new graph
workflow = StateGraph(State)

# Define the conversation node and the summarize node
workflow.add_node("conversation", call_model)
workflow.add_node(summarize_conversation)

# Set the entrypoint as conversation
workflow.add_edge(START, "conversation")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `conversation`.
    # This means these are the edges taken after the `conversation` node is called.
    "conversation",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
)

# We now add a normal edge from `summarize_conversation` to END.
# This means that after `summarize_conversation` is called, we end.
workflow.add_edge("summarize_conversation", END)

# Finally, we compile it!
app = workflow.compile(checkpointer=memory)

In [26]:
def print_update(update):
    for k, v in update.items():
        for m in v["messages"]:
            m.pretty_print()
        if "summary" in v:
            print(v["summary"])

In [27]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "4"}}
input_message = HumanMessage(content="hi! I'm bob")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

input_message = HumanMessage(content="what's my name?")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

input_message = HumanMessage(content="i like the celtics!")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

hi! I'm bob
================================== Ai Message ==================================

It's nice to meet you, Bob! I'm an AI assistant created by Anthropic. How can I help you today?
================================ Human Message =================================

what's my name?
================================== Ai Message ==================================

Your name is Bob, as you told me at the beginning of our conversation.
================================ Human Message =================================

i like the celtics!
================================== Ai Message ==================================

That's great, the Celtics are a fun team to follow! Basketball is an exciting sport. Do you have a favorite Celtics player or a favorite moment from a Celtics game you've watched? I'd be happy to discuss the team and the sport with you.


We can see that so far no summarization has happened - this is because there are only six messages in the list.

In [28]:
values = app.get_state(config).values
values

{'messages': [HumanMessage(content="hi! I'm bob", id='6534853d-b8a7-44b9-837b-eb7abaf7ebf7'),
  AIMessage(content="It's nice to meet you, Bob! I'm an AI assistant created by Anthropic. How can I help you today?", response_metadata={'id': 'msg_015wCFew2vwMQJcpUh2VZ5ah', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 12, 'output_tokens': 30}}, id='run-0d33008b-1094-4f5e-94ce-293283fc3024-0'),
  HumanMessage(content="what's my name?", id='0a4f203a-b95a-42a9-b1c5-bb20f68b3251'),
  AIMessage(content='Your name is Bob, as you told me at the beginning of our conversation.', response_metadata={'id': 'msg_01PLp8wg2xDsJbNR9uCtxcGz', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 50, 'output_tokens': 19}}, id='run-3815dd4d-ee0c-4fc2-9889-f6dd40325961-0'),
  HumanMessage(content='i like the celtics!', id='ac128172-42d1-4390-b7cc-7bcb2d22ee48'),
  AIMessage(content="That

Now let's send another message in

In [29]:
input_message = HumanMessage(content="i like how much they win")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

i like how much they win
================================== Ai Message ==================================

That's understandable, the Celtics have been one of the more successful NBA franchises over the years. Their history of winning championships is very impressive. It's always fun to follow a team that regularly competes for titles. What do you think has been the key to the Celtics' sustained success? Is there a particular era or team that stands out as your favorite?
================================ Remove Message ================================


================================ Remove Message ================================


================================ Remove Message ================================


================================ Remove Message ================================


================================ Remove Message ================================


===========================

If we check the state now, we can see that we have a summary of the conversation, as well as the last two messages

In [30]:
values = app.get_state(config).values
values

{'messages': [HumanMessage(content='i like how much they win', id='bb916ce7-534c-4d48-9f92-e269f9dc4859'),
  AIMessage(content="That's understandable, the Celtics have been one of the more successful NBA franchises over the years. Their history of winning championships is very impressive. It's always fun to follow a team that regularly competes for titles. What do you think has been the key to the Celtics' sustained success? Is there a particular era or team that stands out as your favorite?", response_metadata={'id': 'msg_01B7TMagaM8xBnYXLSMwUDAG', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 148, 'output_tokens': 82}}, id='run-c5aa9a8f-7983-4a7f-9c1e-0c0055334ac1-0')],
 'summary': "Here is a summary of our conversation so far:\n\n- You introduced yourself as Bob and said you like the Boston Celtics basketball team.\n- I acknowledged that it's nice to meet you, Bob, and noted that you had shared your name earlier in th

We can now resume having a conversation! Note that even though we only have the last two messages, we can still ask it questions about things mentioned earlier in the conversation (because we summarized those)

In [31]:
input_message = HumanMessage(content="what's my name?")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

what's my name?
================================== Ai Message ==================================

In our conversation so far, you introduced yourself as Bob. I acknowledged that earlier when you had shared your name.


In [32]:
input_message = HumanMessage(content="what NFL team do you think I like?")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

what NFL team do you think I like?
================================== Ai Message ==================================

I don't actually have any information about what NFL team you might like. In our conversation so far, you've only mentioned that you're a fan of the Boston Celtics basketball team. I don't have any prior knowledge about your preferences for NFL teams. Unless you provide me with that information, I don't have a basis to guess which NFL team you might be a fan of.


In [33]:
input_message = HumanMessage(content="i like the patriots!")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

i like the patriots!
================================== Ai Message ==================================

Okay, got it! Thanks for sharing that you're also a fan of the New England Patriots in the NFL. That makes sense, given your interest in other Boston sports teams like the Celtics. The Patriots have also had a very successful run over the past couple of decades, winning multiple Super Bowls. It's fun to follow winning franchises like the Celtics and Patriots. Do you have a favorite Patriots player or moment that stands out to you?
================================ Remove Message ================================


================================ Remove Message ================================


================================ Remove Message ================================


================================ Remove Message ================================


================================ Remove Message 